In [9]:
# coding: utf-8
"""
simrankを実装するためのコード
"""
from igraph import *
import csv
import collections
import pickle
import numpy as np
from openopt import QP

In [38]:
def simrank(G,C,n,t=10):
    S = np.identity(n)
    I = np.identity(n)
    G = normalize(G)
    i = 1
    while True:
        S = C * np.dot(np.dot(G.T,S),G) + (1-C) * I
        for j in range(n):
            S[j][j] = 1
        if i >= t:
            break
        i += 1
    return S

def normalize(G):
    s = G.sum(0)
    return G/s

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def readdump(path):
    f = open(path, "r")
    arr = pickle.load(f)
    f.close()
    return arr

# 有向エッジリストを入力して、matrixを返す
def cal_edgelist_to_matrix(list_edge):
    # ノードリスト
    list_vertices = list(set([word for row in list_edge for word in row]))
    dict_vertex_to_num = {vertex: i for i, vertex in enumerate(list_vertices)}
    dict_num_to_vertex = {value: key for key, value in dict_vertex_to_num.items()}
    M = np.zeros((len(dict_vertex_to_num), len(dict_vertex_to_num)))
    for vertex1, vertex2 in list_edge:
        M[dict_vertex_to_num[vertex1]][dict_vertex_to_num[vertex2]] += 1
    return M, dict_vertex_to_num, dict_num_to_vertex

In [39]:
# エッジリストの読み込み
list_edge = readcsv("./files/kaigo_honne/list_edgelist20160127.csv")
# 元のネットワークを作成する（無向）
M, dict_vertex_to_num, dict_num_to_vertex = cal_edgelist_to_matrix(list_edge)

In [55]:
C = 0.8
n = 1073
S = simrank(M, C, n, t=200)

In [56]:
dict_vertex_to_num['料理']

851

In [57]:
list_num_score = [(i, row) for i, row in enumerate(S[851])]
list_num_score = sorted(list_num_score, key=lambda x: x[1], reverse=True)

In [58]:
for i in range(10):
    print dict_num_to_vertex[list_num_score[i][0]], list_num_score[i][1]

料理 1.0
要素 0.0417369497579
床暖房 0.0391652768272
遊び 0.0328150300985
京王 0.0304979305976
想定内 0.0304564135453
活き活き 0.0302797718605
サイズ 0.0286641214816
売店 0.0262673172453
記載 0.0260635401995


In [54]:
for i in range(10):
    print dict_num_to_vertex[list_num_score[i][0]], list_num_score[i][1]

料理 1.0
要素 0.0417369497565
床暖房 0.0391652768258
遊び 0.032815030097
京王 0.0304979305961
想定内 0.0304564135439
活き活き 0.030279771859
サイズ 0.0286641214801
売店 0.0262673172439
記載 0.026063540198
